In [27]:
# Part 1 - Building the CNN
#importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
import h5py

#Input Image Size
sz=300

In [28]:
# Initialing the CNN
classifier = Sequential()

In [29]:
# Step 1 - Convolutio Layer 
classifier.add(Convolution2D(32, (3,3) , input_shape = (sz, sz,1), activation = 'relu', padding='same'))

In [30]:
#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [31]:
# Adding second convolution layer
classifier.add(Convolution2D(32, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [32]:
#Adding 3rd Concolution Layer
classifier.add(Convolution2D(64, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))


In [33]:
#Step 3 - Flattening
classifier.add(Flatten())

In [34]:
#Step 4 - Full Connection
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(35, activation = 'softmax')) # 36 Units are the value of no of Classes

In [35]:
#Compiling The CNN
classifier.compile(
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [36]:
# Classifier Summary
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 300, 300, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 150, 150, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 37, 37, 64)      

In [37]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [38]:
#Dataset Store in Variable
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'C:/Users/amits/OneDrive/Desktop/project/Sign-Language-Recognition-Jayu/marathi_module/data/train',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'C:/Users/amits/OneDrive/Desktop/project/Sign-Language-Recognition-Jayu/marathi_module/data/test',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical')

Found 10589 images belonging to 35 classes.
Found 2450 images belonging to 35 classes.


In [39]:
#Model Initialize
epochs=25
model = classifier.fit_generator(
        training_set,
        steps_per_epoch=330,	#No of Image in Traing Set 7000/batch size(32)=218
        epochs=epochs, #Previous value is 25
        validation_data = test_set,
        validation_steps =76 	#No of Image in Test Set 1750/batch size(32)=54
      )

C:\Users\amits\AppData\Local\Temp\ipykernel_3864\2771635669.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = classifier.fit_generator(


Epoch 1/25
330/330 [==============================] - 476s 1s/step - loss: 1.2331 - accuracy: 0.6635 - val_loss: 1.9757 - val_accuracy: 0.6830
Epoch 2/25
330/330 [==============================] - 458s 1s/step - loss: 0.2547 - accuracy: 0.9224 - val_loss: 1.9798 - val_accuracy: 0.6727
Epoch 3/25
330/330 [==============================] - 444s 1s/step - loss: 0.1432 - accuracy: 0.9558 - val_loss: 1.9317 - val_accuracy: 0.6986
Epoch 4/25
330/330 [==============================] - 433s 1s/step - loss: 0.1012 - accuracy: 0.9684 - val_loss: 2.1554 - val_accuracy: 0.7541
Epoch 5/25
330/330 [==============================] - 451s 1s/step - loss: 0.0847 - accuracy: 0.9739 - val_loss: 1.8126 - val_accuracy: 0.7627
Epoch 6/25
330/330 [==============================] - 446s 1s/step - loss: 0.0543 - accuracy: 0.9828 - val_loss: 1.3963 - val_accuracy: 0.7821
Epoch 7/25
330/330 [==============================] - 445s 1s/step - loss: 0.0521 - accuracy: 0.9829 - val_loss: 1.7718 - val_accuracy: 0.7377

In [40]:
#Saving the model
classifier.save('Trained_model.h5')

# model_json = classifier.to_json()
# with open("/content/drive/MyDrive/Data/model-bw1.json", "w") as json_file:
#     json_file.write(model_json)
print('Model Saved')

print(model.history.keys())

Model Saved
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [41]:
#Accuracy
model.history['accuracy']

[0.663540780544281,
 0.9224211573600769,
 0.9557639360427856,
 0.9683622121810913,
 0.9738562107086182,
 0.9827602505683899,
 0.982854962348938,
 0.9856019616127014,
 0.9843705892562866,
 0.9878753423690796,
 0.9896751046180725,
 0.9899592399597168,
 0.9913801550865173,
 0.9927062392234802,
 0.9929904341697693,
 0.9929904341697693,
 0.9917590022087097,
 0.9938429594039917,
 0.9948849081993103,
 0.9942218661308289,
 0.9953585267066956,
 0.9951691031455994,
 0.994411289691925,
 0.9936534762382507,
 0.9956427216529846]